In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [1]:
import os
os.getcwd()
os.chdir('c:\\Users\\Stior\\Documents\\cours\\L3SID\\Projet-InterPromo\
\\ProjetInterpromo2020G8')

In [2]:
from pipeline import Pipeline, Process, Postprocess, Preprocess
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import PIL

In [3]:
class MyPreProcess(Preprocess):
    process_desc = "Exemple de pre-process -> ne fait rien"
    def run(self, images):
        pass


class MyProcess(Process):
    process_desc = "Exemple de process -> ne fait rien"
    def run(self, images):
        pass
    

class MyPostProcess(Postprocess):
    process_desc = "Exemple de post-process -> ne fait rien"
    def run(self, images):
        pass

In [4]:
pipeline = Pipeline()
process_array = np.array([MyPreProcess()
                          ,MyProcess()
                          ,MyPostProcess()
                          ,MyPreProcess(verbose=0)])
pipeline.add_processes(process_array)


Preprocess :  Exemple de pre-process -> ne fait rien
Process :  Exemple de process -> ne fait rien
Postprocess :  Exemple de post-process -> ne fait rien
Exemple de pre-process -> ne fait rien a été ajouté.
Exemple de process -> ne fait rien a été ajouté.
Exemple de post-process -> ne fait rien a été ajouté.
Exemple de pre-process -> ne fait rien a été ajouté.


In [5]:
pipeline.print_process()

Exemple de pre-process -> ne fait rien
Exemple de pre-process -> ne fait rien
Exemple de process -> ne fait rien
Exemple de post-process -> ne fait rien


### Ouverture d'image

In [6]:
img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 0)
img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 1)

# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/\
#                         Aer_Lingus_Airbus_A320_plane9.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/\
#                         Aer_Lingus_Airbus_A320_plane9.jpg', 1)


# # img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/\
#                         Air_Transat_Airbus_A330-300_V2_plane128.jpg', 0)
# # img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/\
#                         Air_Transat_Airbus_A330-300_V2_plane128.jpg', 1)

# template = cv2.imread('./images/temp_normal_seat.jpg', 0)
# template = cv2.imread('./images/templateTest.jpg', 0)
# template = cv2.imread('./images/templateNormalSeat-SansBlanc.png', 0)
# template = cv2.imread('./images/templateNormalSeat-SansBlanc.jpg', 0)
template = cv2.imread('./images/templateTest-sansBlanc.png', 0)

# cv2.imshow('Test', img2)
# cv2.waitKey()
# cv2.destroyAllWindows()
h, w = template.shape
# Match template method
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',\
'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

### Premier test de pattern matching + sortie en txt

In [8]:

res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
THRESHOLD = 0.8
loc = np.where( res >= THRESHOLD)

coord = []

for pt in zip(*loc[::-1]):
    coord.append((pt[0], pt[1]))
    cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

for i in coord:
    cv2.circle(img_rgb, i, 1, (255, 0, 0), -1)

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows() 

print(coord)


with open("coord.txt", "a") as fichier:
    for xy in coord:
        fichier.write(str(xy[0]) + ", " + str(xy[1]) + "\n")



[(146, 517), (133, 537), (146, 537), (185, 537), (198, 537), (211, 537), (224, 537), (263, 537), (276, 537), (133, 557), (146, 557), (185, 557), (198, 557), (211, 557), (224, 557), (263, 557), (276, 557), (133, 577), (146, 577), (185, 577), (198, 577), (211, 577), (224, 577), (263, 577), (276, 577), (133, 597), (146, 597), (185, 597), (198, 597), (211, 597), (224, 597), (263, 597), (276, 597), (133, 617), (146, 617), (185, 617), (198, 617), (211, 617), (224, 617), (263, 617), (276, 617), (133, 637), (146, 637), (185, 637), (198, 637), (211, 637), (224, 637), (263, 637), (276, 637), (133, 657), (146, 657), (185, 657), (198, 657), (211, 657), (224, 657), (263, 657), (276, 657), (133, 677), (146, 677), (185, 677), (198, 677), (211, 677), (224, 677), (263, 677), (276, 677), (133, 697), (146, 697), (185, 697), (198, 697), (211, 697), (224, 697), (263, 697), (276, 697), (133, 717), (146, 717), (185, 717), (198, 717), (211, 717), (224, 717), (263, 717), (276, 717), (133, 737), (146, 737), (18

### Second test avec list de pattern

In [9]:
img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 0)
img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Aer_Lingus_Airbus_A330-200_plane4.jpg', 1)
# template = cv2.imread('./images/temp_normal_seat.jpg', 0)
templates = ['./images/temp_business_seat_armrest_left.jpg', './images/temp_business_seat_armrest_right.jpg', './images/temp_business_elec1.jpg']
# templates = ['./images/templateBusinessElectricity.jpg']

for templ in templates:
    template = cv2.imread(templ, 0)
    h, w = template.shape
    res = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
    if templ == './images/temp_business_elec1.jpg':
        THRESHOLD = 0.7
    else:
        THRESHOLD = 0.7

    loc = np.where(res >= THRESHOLD)

    coord = []

    fichier = open("coord.txt", "a")

    if templ == './images/temp_business_elec1.jpg':
        fichier.write('-------------------\nPrise electrique \n')

    for pt in zip(*loc[::-1]):
        fichier.write(str(pt[0]) + ", " + str(pt[1]) + "\n")
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
    fichier.close()

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows()
# cv2.imwrite('test.jpeg', img_rgb)

print(np.where(res >= THRESHOLD))

(array([334, 334, 334, 334, 334, 367, 367, 368, 399, 401, 431, 431, 435,
       435, 435], dtype=int64), array([140, 190, 217, 268, 269, 191, 218, 250, 190, 140, 191, 218, 149,
       250, 277], dtype=int64))


In [45]:
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-900_plane259.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-900_plane259.jpg', 1) # Ok
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-1000_plane258.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A350-1000_plane258.jpg', 1) # Ok
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A330_33E_plane255.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Cathay_Pacific_Airbus_A330_33E_plane255.jpg', 1) # Ok
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/American_Airlines_777-300_ER_plane167.svg.png', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/American_Airlines_777-300_ER_plane167.svg.png', 1) # Null
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Asiana_Boeing_777-200_ER_D_plane206.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Asiana_Boeing_777-200_ER_D_plane206.jpg', 1) # Null - rectangle autour
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Avianca_Boeing_787-8_plane313.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/Avianca_Boeing_787-8_plane313.jpg', 1) #Ok
img_gray = cv2.imread('./theo_remove_caracters_yellow.jpg', 0)
img_rgb = cv2.imread('./theo_remove_caracters_yellow.jpg', 1) #Ok


# template = cv2.imread('./images/temp_normal_seat.jpg', 0)
templates = ['./images/temp_business_turn_left.jpg', 'templateTest.jpg', './images/templateNormalSeat-SansBlanc.jpg', './images/temp_normal_seat_elec.jpg']




#### Rotation d'un template + changement du noir en blanc

In [32]:
# Rotation 45° left + black => white

templateTest = PIL.Image.open('./images/temp_business_seat_footrest.jpg')
rotate = templateTest.rotate(45)
rotate = rotate.convert('RGBA')

data = np.array(rotate)   # "data" is a height x width x 4 numpy array
red, green, blue, alpha = data.T # Temporarily unpack the bands for readability


black_areas = (red == 0) & (blue == 0) & (green == 0)
data[..., :-1][black_areas.T] = (255, 255, 255) # Black back to white
rotate = PIL.Image.fromarray(data)
rotate.show()
rotate = rotate.convert('RGB')
rotate.save('templateTest.jpg')

#### Transformation d'un template en son symétrique par rapport a la droite

In [46]:
# Right symmetry
templateTest = PIL.Image.open('./images/temp_business_turn_left.jpg')
rotate = templateTest.transpose(PIL.Image.FLIP_LEFT_RIGHT)

rotate.show()
rotate.save('templateTest.jpg')


#107-171-93

#### Changement du fond blanc => couleur voulu

In [23]:
templateTest = PIL.Image.open(templates[2])
templateTest = templateTest.convert('RGBA')

data = np.array(templateTest)
red, green, blue, alpha = data.T

THRESHOLD = 250 # To chose white background

white_areas = (red >= THRESHOLD) & (blue >= THRESHOLD) & (green >= THRESHOLD)
data[..., :-1][white_areas.T] = (255, 0, 0) # Set background color
templateTest = PIL.Image.fromarray(data)
templateTest.show()

#### Test avec les pattern précédement modifié + sortie des coordonnées pour chaque siege

In [47]:
for templ in templates:
    template = cv2.imread(templ, 0)
    h, w = template.shape
    
    res = cv2.matchTemplate(img_gray, template,cv2.TM_CCOEFF_NORMED)
    
    THRESHOLD = 0.9
    if templ == 'templateTest.jpg':
        THRESHOLD = 0.60
    if templ == templates[0]:
        THRESHOLD = 0.60
    if templ == templates[3]:
        THRESHOLD = 0.7


    loc = np.where( res >= THRESHOLD)

    # fichier = open("coord.txt", "a")
    for pt in zip(*loc[::-1]):
        # fichier.write(str(pt[0]) + ", " + str(pt[1]) + "\n")
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)

# fichier.close()

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.imwrite('test.jpg', img_rgb)

#print(np.where( res >= THRESHOLD))

True

### Mise au propre + commmentaire + test sur plusieur image similaire

In [41]:
# Input Image plane
# Ok
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'Cathay_Pacific_Airbus_A350-900_plane259.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'Cathay_Pacific_Airbus_A350-900_plane259.jpg', 1)
# OK
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'Cathay_Pacific_Airbus_A350-1000_plane258.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'Cathay_Pacific_Airbus_A350-1000_plane258.jpg', 1)
# Ok
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'Cathay_Pacific_Airbus_A330_33E_plane255.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'Cathay_Pacific_Airbus_A330_33E_plane255.jpg', 1)
# NOk
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'American_Airlines_777-300_ER_plane167.svg.png', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'American_Airlines_777-300_ER_plane167.svg.png', 1)
# NOk
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'Asiana_Boeing_777-200_ER_D_plane206.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'Asiana_Boeing_777-200_ER_D_plane206.jpg', 1)
# OK
# img_gray = cv2.imread('../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/'\
# 'Avianca_Boeing_787-8_plane313.jpg', 0)
# img_rgb = cv2.imread('../All Data/ANALYSE IMAGE/' \
# 'LAYOUT SEATGURU/Avianca_Boeing_787-8_plane313.jpg', 1)
# Ok
img_gray = cv2.imread('./theo_remove_caracters_yellow.jpg', 0)
img_rgb = cv2.imread('./theo_remove_caracters_yellow.jpg', 1)

# Input template
templates = [
            './images/temp_business_turn_left.jpg',
            './images/temp_business_turn_right.jpg',
            './images/temp_normal_seat_elec.jpg'
            ]

for templ in templates:
    # Open template
    template = cv2.imread(templ, 0)
    h, w = template.shape

    # List of match
    res = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)

    # Set accuracy for each template
    threshold = 0.9
    if templ == templates[0]:
        print('temp_business_turn_left')
        threshold = 0.60
    if templ == templates[1]:
        print('temp_business_turn_right')
        threshold = 0.60
    if templ == templates[2]:
        print('temp_normal_seat_elec')
        threshold = 0.63

    loc = np.where(res >= threshold)  # Filtering with the previous THRESHOLD

    for pt in zip(*loc[::-1]):
        # Output seat position
        print(str(pt[0]) + ', ' + str(pt[1]))
        # Tracing rectangle
        cv2.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)

# Display results
# cv2.imshow('test', img_rgb)
# cv2.waitKey()
# cv2.destroyAllWindows()

# Save results
# cv2.imwrite('test.jpg', img_rgb)

temp_business_turn_left
216, 274
215, 307
216, 307
215, 308
216, 308
135, 312
215, 341
216, 341
216, 342
135, 346
215, 374
215, 375
216, 375
216, 376
135, 379
135, 380
215, 408
216, 408
215, 409
216, 409
135, 413
136, 413
135, 414
215, 442
216, 442
216, 443
135, 446
135, 447
215, 475
216, 475
215, 476
216, 476
135, 480
135, 481
215, 509
216, 509
216, 510
135, 513
135, 514
135, 643
216, 646
135, 676
135, 677
136, 677
135, 678
215, 679
216, 679
215, 680
216, 680
temp_business_turn_right
176, 307
177, 307
176, 308
177, 308
257, 312
176, 341
177, 341
176, 342
177, 342
257, 346
258, 346
177, 374
176, 375
177, 375
176, 376
257, 379
258, 379
257, 380
176, 408
177, 408
176, 409
177, 409
257, 413
258, 413
257, 414
176, 442
177, 442
176, 443
257, 446
258, 446
257, 447
258, 447
176, 475
177, 475
176, 476
177, 476
257, 480
258, 480
257, 481
176, 509
177, 509
176, 510
257, 513
257, 514
258, 514
257, 643
257, 676
258, 676
257, 677
258, 677
257, 678
176, 679
177, 679
176, 680
177, 680
temp_normal_sea

### Creation d'une fonction du code précédent : 
#### Entrée : chemin d'image, liste de chemin de templates, liste de seuils
#### Sortie : Liste de liste de coordonnée pour chaque template

In [9]:
def coord_seats_finder(path_image, templates = ['./images/temp_business_turn_left.jpg', './images/temp_business_turn_right.jpg', './images/temp_normal_seat_elec.jpg'], thresholds = [0.60, 0.60, 0.63], show = False):
    """
    input:
        path_image : image plane
        templates : list of templates for pattern matching
        thresholds : list of threshold for each template, set accuracy
    output:
        position : list of list right angle position for each pattern match
    """
    # Open image
    img_rgb = cv2.imread(path_image, 1)
    img_gray = cv2.imread(path_image, 0)
    position = []
    for i in range(len(templates)):  # Iteration on each pattern
        template = cv2.imread(templates[i], 0)  # Read pattern image
        h, w = template.shape
        # List of match
        res = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
        position.append([])
        for pos in zip(*np.where(res >= thresholds[i])[::-1]):
            position[i].append(pos)  # add position to output
            cv2.rectangle(img_rgb, pos, (pos[0] + w, pos[1] + h), (0, 0, 255), 2)
    if show:
        cv2.imshow('test', img_rgb)
        cv2.waitKey()
        cv2.destroyAllWindows()
    return(position)

print(coord_seats_finder('./theo_remove_caracters_yellow.jpg', show = False))


[[(216, 274), (215, 307), (216, 307), (215, 308), (216, 308), (135, 312), (215, 341), (216, 341), (216, 342), (135, 346), (215, 374), (215, 375), (216, 375), (216, 376), (135, 379), (135, 380), (215, 408), (216, 408), (215, 409), (216, 409), (135, 413), (136, 413), (135, 414), (215, 442), (216, 442), (216, 443), (135, 446), (135, 447), (215, 475), (216, 475), (215, 476), (216, 476), (135, 480), (135, 481), (215, 509), (216, 509), (216, 510), (135, 513), (135, 514), (135, 643), (216, 646), (135, 676), (135, 677), (136, 677), (135, 678), (215, 679), (216, 679), (215, 680), (216, 680)], [(176, 307), (177, 307), (176, 308), (177, 308), (257, 312), (176, 341), (177, 341), (176, 342), (177, 342), (257, 346), (258, 346), (177, 374), (176, 375), (177, 375), (176, 376), (257, 379), (258, 379), (257, 380), (176, 408), (177, 408), (176, 409), (177, 409), (257, 413), (258, 413), (257, 414), (176, 442), (177, 442), (176, 443), (257, 446), (258, 446), (257, 447), (258, 447), (176, 475), (177, 475), 

### Simplification de la fonction coord_seats_finder() :
#### Entrée : une image, un template, un seuil
#### Sortie : list de coordonnée

In [10]:
def coord_pattern_finder(image, template, threshold: float):
    """
    input:
        image : image plane cv2.imread() black and white
        template : image pattern cv2.imread() black and white
        threshold : threshold for this pattern
    output:
        position : list right angle position for this pattern on the image

    """
    position = []  # Variable output
    # List of match
    res = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    for pos in zip(*np.where(res >= threshold)[::-1]):
        position.append(pos)
    return(position)

### Test de la fonction coord_pattern_finder()

In [21]:
path_image = './theo_remove_caracters_yellow.jpg'
templates = ['./images/temp_business_turn_left.jpg', './images/temp_business_turn_right.jpg', './images/temp_normal_seat_elec.jpg']
thresholds = [0.60, 0.60, 0.63]
img = cv2.imread(path_image, 0)
template = cv2.imread(templates[0], 0)
h, w = template.shape
img_rgb = cv2.imread(path_image, 1)

position = coord_pattern_finder(img, template, 0.6)

for pos in position:
    cv2.rectangle(img_rgb, pos, (pos[0] + w, pos[1] + h), (0, 0, 255), 2)

cv2.imshow('test', img_rgb)
cv2.waitKey()
cv2.destroyAllWindows()

### Test sur plusieur plan ou famille de plan avec choix d'un ou plusieur pattern

In [25]:
pathTempl = "./Images/" # Path for template
imagesTemp = os.listdir(pathTempl)
templates = []
tempName = 'temp_' # Template or family name

pathPlane = "../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/" # Path for plane images
imagesPlane = os.listdir(pathPlane)
plane = []

planeName = 'Aer_Lingus_' # Plane or family name


for i in imagesTemp:
    if 'temp_' in i and 'seat' in i:
        templates.append(i)

for i in imagesPlane:
    if planeName in i:
        plane.append(i)

for p in plane:
    img_rgb = cv2.imread(pathPlane + p, 1)
    img_gray = cv2.imread(pathPlane + p, 0)
    print(p + ' shape', img_gray.shape)

    # fichier = open("coord_" + p + ".txt", "a")
    
    for templ in templates:
        template = cv2.imread(pathTempl + templ, 0)
        h, w = template.shape

        threshold = 0.8 #  Default threshold
        # Choose threshold for each template
        if templ == 'temp_business_seat_armrest_left.jpg':
            threshold = 0.6
        elif templ == 'temp_business_seat_armrest_right.jpg':
            threshold = 0.6
        elif templ == 'temp_business_seat_footrest.jpg':
            threshold = 1
        elif templ == 'temp_first_seat.jpg':
            threshold = 1
        elif templ == 'temp_normal_seat.png':
            threshold = 0.7
        elif templ == 'temp_normal_seat_elec.jpg':
            threshold = 1
        elif templ == 'temp_normal_seat_little.jpg':
            threshold = 0.65
        elif templ == 'temp_yellow_seat.jpg':
            threshold = 1
        threshold = 0.8


        position = coord_pattern_finder(img_gray, template, threshold)
        
        # fichier.write(templ + '\n')
        for pos in position:
            # fichier.write(str(pos[0]) + ', ' + str(pos[1]) + '\n')
            cv2.rectangle(img_rgb, pos, (pos[0] + w, pos[1] + h), (0, 0, 255), 2)

    # fichier.close()
    # cv2.imshow(p, img_rgb)
    # cv2.waitKey()
    # cv2.destroyAllWindows()

Aer_Lingus_Airbus_A320_plane9.jpg shape (1657, 425)
Aer_Lingus_Airbus_A321_plane1.jpg shape (1944, 425)
Aer_Lingus_Airbus_A321_plane10.jpg shape (1944, 425)
Aer_Lingus_Airbus_A330-200_B_plane5.jpg shape (1564, 425)
Aer_Lingus_Airbus_A330-200_B_plane7.jpg shape (1564, 425)
Aer_Lingus_Airbus_A330-200_plane4.jpg shape (1546, 425)
Aer_Lingus_Airbus_A330-200_plane6.jpg shape (1546, 425)
Aer_Lingus_Airbus_A330-300_A_plane6.jpg shape (1764, 425)
Aer_Lingus_Airbus_A330-300_A_plane8.jpg shape (1764, 425)
Aer_Lingus_Boeing_757-200_plane11.jpg shape (1734, 425)
Aer_Lingus_Boeing_757-200_plane2.jpg shape (1734, 425)


### Recupération nombre de seat par type d'avion et par catégorie

In [34]:
pathPlane = "../All Data/ANALYSE IMAGE/LAYOUT SEATGURU/" # Path for plane images
imagesPlane = os.listdir(pathPlane)
plane = []
planeName = 'Aer_Lingus_' # Plane or family name

for i in imagesPlane:
    if planeName in i:
        plane.append(i)
print(plane)

SEATGURU_INFO_AIRCRAFT = pd.read_csv('../All data/SEATGURU_INFO_AIRCRAFT.csv', sep=';')
image_lignes = pd.read_csv('Image_lignes.csv', sep=';')

nbSeatTotal = {}

for rowNumber, data in image_lignes.iterrows():
    if data['image'] in p:
        for ligne in data['lignes']:
            if not (data['image'] in nbSeatTotal.keys()):
                nbSeatTotal[data['image']] = {}
                nbSeatTotal[data['image']]['Total_seat'] = SEATGURU_INFO_AIRCRAFT[i:int(i)+1]['Total_seat']

            


# for rowNumber, data in SEATGURU_INFO_AIRCRAFT[0:1000].iterrows():
#     if data['Airline_name'] == 'aer-lingus':
#         if not ('Aer_Lingus_' + data['Aircraft_Type'] in nbSeatTotal.keys()):
#             nbSeatTotal['Aer_Lingus_' + data['Aircraft_Type']] = {}  # Create dict for each category of seat
#             nbSeatTotal['Aer_Lingus_' + data['Aircraft_Type']]['Total_seat'] = data['Total_seat']  # insert Total seats number on dict
#         if not (data['Category'] in nbSeatTotal['Aer_Lingus_' + data['Aircraft_Type']]):
#             nbSeatTotal['Aer_Lingus_' + data['Aircraft_Type']][data['Category']] = data['Count']  # insert seats for each category

# Aer_Lingus_Airbus_A321_plane1

print(nbSeatTotal)


['Aer_Lingus_Airbus_A320_plane9.jpg', 'Aer_Lingus_Airbus_A321_plane1.jpg', 'Aer_Lingus_Airbus_A321_plane10.jpg', 'Aer_Lingus_Airbus_A330-200_B_plane5.jpg', 'Aer_Lingus_Airbus_A330-200_B_plane7.jpg', 'Aer_Lingus_Airbus_A330-200_plane4.jpg', 'Aer_Lingus_Airbus_A330-200_plane6.jpg', 'Aer_Lingus_Airbus_A330-300_A_plane6.jpg', 'Aer_Lingus_Airbus_A330-300_A_plane8.jpg', 'Aer_Lingus_Boeing_757-200_plane11.jpg', 'Aer_Lingus_Boeing_757-200_plane2.jpg']


ValueError: invalid literal for int() with base 10: 'Xiamen_Airlines_Boeing_787-9_plane342.jpg'

In [32]:
SEATGURU_INFO_AIRCRAFT = pd.read_csv('../All data/SEATGURU_INFO_AIRCRAFT.csv', sep=';')

SEATGURU_INFO_AIRCRAFT[100:101]['Aircraft_Type']


100    Russian Airlines Airbus A330 300 V3
Name: Aircraft_Type, dtype: object

### Creation category template

In [18]:
def templ_category(path = './images/', category = 'business', ):
    """
    Input:
        Path: directory path of templates
        category: name of category
    Output:
        templates: list of template name
    """
    imagesTemp = os.listdir(path)
    templates = []

    for i in imagesTemp:
        if 'temp_' + category in i:
            templates.append(i)
    
    return(templates)

print(templ_category(category = 'normal'))

['temp_normal_seat.jpg', 'temp_normal_seat.png', 'temp_normal_seat_a.jpg', 'temp_normal_seat_b.jpg', 'temp_normal_seat_c.jpg', 'temp_normal_seat_d.jpg', 'temp_normal_seat_e.jpg', 'temp_normal_seat_elec.jpg', 'temp_normal_seat_f.jpg', 'temp_normal_seat_little.jpg', 'temp_normal_seat_red.jpg']


### calcul taux réussite

In [25]:
templ_business = templ_category(category = 'business')
templ_normal = templ_category(category = 'normal')

for i in plane:
    if 






SyntaxError: invalid syntax (<ipython-input-25-326b9de040bc>, line 15)